In [1]:
import re
import nltk
import numpy as np
import pandas as pd 
from nltk.corpus import stopwords
from tokenizers import Tokenizer
from IPython.display import clear_output
from warnings import filterwarnings
filterwarnings("ignore")

C:\Users\Ahmed\AppData\Local\Temp\ipykernel_7812\4226171187.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Read the data 

In [2]:
df=pd.read_csv(r'fake_job_postings.csv')
df.head()

job_id                                      title            location  \
0       1                           Marketing Intern    US, NY, New York   
1       2  Customer Service - Cloud Video Production      NZ, , Auckland   
2       3    Commissioning Machinery Assistant (CMA)       US, IA, Wever   
3       4          Account Executive - Washington DC  US, DC, Washington   
4       5                        Bill Review Manager  US, FL, Fort Worth   

  department salary_range                                    company_profile  \
0  Marketing          NaN  We're Food52, and we've created a groundbreaki...   
1    Success          NaN  90 Seconds, the worlds Cloud Video Production ...   
2        NaN          NaN  Valor Services provides Workforce Solutions th...   
3      Sales          NaN  Our passion for improving quality of life thro...   
4        NaN          NaN  SpotSource Solutions LLC is a Global Human Cap...   

                                         description  \
0  Food52, a fast-growing, James Beard Award-winn...   
1  Organised - Focused - Vibrant - Awesome!Do you...   
2  Our client, located in Houston, is actively se...   
3  THE COMPANY: ESRI – Environmental Systems Rese...   
4  JOB TITLE: Itemization Review ManagerLOCATION:...   

                                        requirements  \
0  Experience with content management systems a m...   
1  What we expect from you:Your key responsibilit...   
2  Implement pre-commissioning and commissioning ...   
3  EDUCATION: Bachelor’s or Master’s in GIS, busi...   
4  QUALIFICATIONS:RN license in the State of Texa...   

                                            benefits  telecommuting  \
0                                                NaN              0   
1  What you will get from usThrough being part of...              0   
2                                                NaN              0   
3  Our culture is anything but corporate—we have ...              0   
4                              Full Benefits Offered              0   

   has_company_logo  has_questions employment_type required_experience  \
0                 1              0           Other          Internship   
1                 1              0       Full-time      Not Applicable   
2                 1              0             NaN                 NaN   
3                 1              0       Full-time    Mid-Senior level   
4                 1              1       Full-time    Mid-Senior level   

  required_education                   industry              function  \
0                NaN                        NaN             Marketing   
1                NaN  Marketing and Advertising      Customer Service   
2                NaN                        NaN                   NaN   
3  Bachelor's Degree          Computer Software                 Sales   
4  Bachelor's Degree     Hospital & Health Care  Health Care Provider   

   fraudulent  
0           0  
1           0  
2           0  
3           0  
4           0

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17880 non-null  int64 
 1   title                17880 non-null  object
 2   location             17534 non-null  object
 3   department           6333 non-null   object
 4   salary_range         2868 non-null   object
 5   company_profile      14572 non-null  object
 6   description          17879 non-null  object
 7   requirements         15184 non-null  object
 8   benefits             10668 non-null  object
 9   telecommuting        17880 non-null  int64 
 10  has_company_logo     17880 non-null  int64 
 11  has_questions        17880 non-null  int64 
 12  employment_type      14409 non-null  object
 13  required_experience  10830 non-null  object
 14  required_education   9775 non-null   object
 15  industry             12977 non-null  object
 16  func

### normal data pre-processing

In [4]:
df['company_profile'].nunique()

1709

In [5]:
df.drop(columns=['job_id','department','salary_range'],inplace=True)

### Data pre-processing

#### Label Encoding

employment_type

In [6]:
df['employment_type'].value_counts()

employment_type
Full-time    11620
Contract      1524
Part-time      797
Temporary      241
Other          227
Name: count, dtype: int64

In [7]:
from sklearn import preprocessing 

label_encoder = preprocessing.LabelEncoder() 

df['employment_type']= label_encoder.fit_transform(df['employment_type'])

In [8]:
df['required_experience'].value_counts()

required_experience
Mid-Senior level    3809
Entry level         2697
Associate           2297
Not Applicable      1116
Director             389
Internship           381
Executive            141
Name: count, dtype: int64

In [9]:
df['required_experience']= label_encoder.fit_transform(df['required_experience'])

In [10]:
df['required_education'].value_counts()

required_education
Bachelor's Degree                    5145
High School or equivalent            2080
Unspecified                          1397
Master's Degree                       416
Associate Degree                      274
Certification                         170
Some College Coursework Completed     102
Professional                           74
Vocational                             49
Some High School Coursework            27
Doctorate                              26
Vocational - HS Diploma                 9
Vocational - Degree                     6
Name: count, dtype: int64

In [11]:
df['required_education']= label_encoder.fit_transform(df['required_education'])

In [12]:
df['fraudulent'].value_counts()

fraudulent
0    17014
1      866
Name: count, dtype: int64

In [13]:
df["function"].nunique()

37

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17880 entries, 0 to 17879
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                17880 non-null  object
 1   location             17534 non-null  object
 2   company_profile      14572 non-null  object
 3   description          17879 non-null  object
 4   requirements         15184 non-null  object
 5   benefits             10668 non-null  object
 6   telecommuting        17880 non-null  int64 
 7   has_company_logo     17880 non-null  int64 
 8   has_questions        17880 non-null  int64 
 9   employment_type      17880 non-null  int32 
 10  required_experience  17880 non-null  int32 
 11  required_education   17880 non-null  int32 
 12  industry             12977 non-null  object
 13  function             11425 non-null  object
 14  fraudulent           17880 non-null  int64 
dtypes: int32(3), int64(4), object(8)
memory usage: 1.8+ M

In [15]:
df.drop(columns=['industry','function'],inplace=True)

In [16]:
df.dropna(inplace=True)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8783 entries, 1 to 17876
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   title                8783 non-null   object
 1   location             8783 non-null   object
 2   company_profile      8783 non-null   object
 3   description          8783 non-null   object
 4   requirements         8783 non-null   object
 5   benefits             8783 non-null   object
 6   telecommuting        8783 non-null   int64 
 7   has_company_logo     8783 non-null   int64 
 8   has_questions        8783 non-null   int64 
 9   employment_type      8783 non-null   int32 
 10  required_experience  8783 non-null   int32 
 11  required_education   8783 non-null   int32 
 12  fraudulent           8783 non-null   int64 
dtypes: int32(3), int64(4), object(6)
memory usage: 857.7+ KB


#### extract label

In [18]:
label=df['fraudulent']
label.head()

1    0
3    0
4    0
6    0
7    0
Name: fraudulent, dtype: int64

### extract features

### Textual data pre-processing 


#### Extract each column

In [19]:
title_sents=df['title']
title_sents.head()

1    Customer Service - Cloud Video Production
3            Account Executive - Washington DC
4                          Bill Review Manager
6                        Head of Content (m/f)
7             Lead Guest Service Specialist   
Name: title, dtype: object

In [20]:
location_sents=df['location']
location_sents.head()

1           NZ, , Auckland
3       US, DC, Washington
4       US, FL, Fort Worth
6           DE, BE, Berlin
7    US, CA, San Francisco
Name: location, dtype: object

In [21]:
company_profile_sents=df['company_profile']
company_profile_sents.head()

1    90 Seconds, the worlds Cloud Video Production ...
3    Our passion for improving quality of life thro...
4    SpotSource Solutions LLC is a Global Human Cap...
6    Founded in 2009, the Fonpit AG rose with its i...
7    Airenvy’s mission is to provide lucrative yet ...
Name: company_profile, dtype: object

In [22]:
describtion_sents=df['description']
describtion_sents.head()

1    Organised - Focused - Vibrant - Awesome!Do you...
3    THE COMPANY: ESRI – Environmental Systems Rese...
4    JOB TITLE: Itemization Review ManagerLOCATION:...
6    Your Responsibilities: Manage the English-spea...
7    Who is Airenvy?Hey there! We are seasoned entr...
Name: description, dtype: object

In [23]:
requirments_sents=df['requirements']
requirments_sents.head()

1    What we expect from you:Your key responsibilit...
3    EDUCATION: Bachelor’s or Master’s in GIS, busi...
4    QUALIFICATIONS:RN license in the State of Texa...
6    Your Know-How:                                ...
7    Experience with CRM software, live chat, and p...
Name: requirements, dtype: object

In [24]:
benefit_sents=df['benefits']
benefit_sents.head()

1    What you will get from usThrough being part of...
3    Our culture is anything but corporate—we have ...
4                                Full Benefits Offered
6    Your Benefits: Being part of a fast-growing co...
7    Competitive Pay. You'll be able to eat steak e...
Name: benefits, dtype: object

#### text pre-processing

### Functions of pre-processing

clean the regex

In [45]:
def clean(tweet):
    tweet=re.sub('(#|@)\w*',"",tweet)# \w [a-z|A_Z|0-9|_]
    tweet=re.sub("https?:\/\/\S+","",tweet)
    tweet=re.sub("(\?|!)+"," ",tweet)
    tweet=re.sub("\s\d+\s","",tweet)
    tweet=re.sub("(\.|\,)+","",tweet)
    tweet=re.sub("^\s+","",tweet)
    tweet=re.sub("\s+$","",tweet)
    return tweet

tokenizing and removing the stopwords 

In [26]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

In [27]:
def rmv_stop_words(sentences):
    return [token for token in sentences if token not in stop_words]

In [43]:
def tokenize(sentences):
    return sentences.split()

lowercase words

In [29]:
def lowercase(sentences):
    return [token.lower() for token in sentences]

Stemming

In [30]:
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

In [31]:
def stemming(sentences):
    return [stemmer.stem(token) for token in sentences]

Clean using all 

In [51]:
def text_processing(sentences):
    return [stemming(rmv_stop_words(lowercase(tokenize(clean(sentence))))) for sentence in sentences]

### Use the Pre-processing functions

In [52]:
title_sents =text_processing(title_sents)

In [53]:
location_sents = text_processing(location_sents)
company_profile_sents=text_processing(company_profile_sents)
describtion_sents=text_processing(describtion_sents)
requirments_sents=text_processing(requirments_sents)
benefit_sents=text_processing(benefit_sents)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [55]:
title_corpus = [' '.join(tweet) for tweet in title_sents]

In [56]:
location_corpus = [' '.join(tweet) for tweet in location_sents]
cp_corpus = [' '.join(tweet) for tweet in company_profile_sents]
d_corpus = [' '.join(tweet) for tweet in describtion_sents]
req_corpus = [' '.join(tweet) for tweet in requirments_sents]
benefits_corpus = [' '.join(tweet) for tweet in benefit_sents]

In [57]:
vectorizer=CountVectorizer()

In [58]:
new_title=vectorizer.fit_transform(title_corpus).toarray()

In [59]:
new_location=vectorizer.fit_transform(location_corpus).toarray()
new_cp=vectorizer.fit_transform(cp_corpus).toarray()
new_desc=vectorizer.fit_transform(d_corpus).toarray()
new_req=vectorizer.fit_transform(req_corpus).toarray()
new_benefits=vectorizer.fit_transform(benefits_corpus).toarray()

In [60]:
from sklearn.model_selection import train_test_split

In [62]:
y=label
y.head()

1    0
3    0
4    0
6    0
7    0
Name: fraudulent, dtype: int64

collect all textual features in x

In [67]:
x=np.concatenate((new_title,new_location,new_cp,new_desc,new_req,new_benefits),axis=1)

In [69]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.2, random_state=42)

## Modeling the data 

In [70]:
from sklearn.linear_model import LogisticRegression
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

LogisticRegression()

In [71]:
from sklearn.metrics import accuracy_score

In [72]:
train_pred = lr_model.predict(X_train)

test_pred = lr_model.predict(X_test)

In [73]:
tr_acc = round(accuracy_score(train_pred, y_train), 4)

ts_acc = round(accuracy_score(test_pred, y_test), 4)

## Evaluating the data 

In [74]:
print(f"Training Accuracy ==> {tr_acc*100}%")
print(f"Testing Accuracy ==> {ts_acc*100}%")

Training Accuracy ==> 100.0%
Testing Accuracy ==> 99.66000000000001%


I think that the reason of this overfitting is 
  -Imbalance data -->fraud is less than not fraud
  